In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Upload the CSV file
from google.colab import files
uploaded = files.upload()

Saving tennis_articles_v4.csv to tennis_articles_v4.csv


In [0]:
# Read the CSV file
import io
df = pd.read_csv(io.StringIO(uploaded['tennis_articles_v4.csv'].decode("utf-8")))

In [5]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [6]:
# split the the text in the articles into sentences
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))  

f=open('tennis.txt','r')
data=f.read()
sentences1 = data.split("\n\n")
'''sentences1 = []
for s in f:
  sentences1.append(sent_tokenize(s))  '''

print(len(sentences),len(sentences1))

8 8


In [8]:
# flatten the list
sentences = [y for x in sentences for y in x]
print(sentences)

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.', "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.", 'I think everyone knows this is my job here.', "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.", "I'm a pretty competitive girl.", "I say my hellos, but I'm not sending any players flowers as well.", "Uhm, I'm not really friendly or close to many players.", "I have not a lot of friends away from the courts.'", 'When she said she is not really close to a lot of players, is that something strategic that she is doing?', "Is it different on the men's tour than the women's tour?", "'No, not at all.", "I thin

In [9]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]
print(clean_sentences)

['maria sharapova has basically no friends as tennis players on the wta tour ', 'the russian player has no problems in openly speaking about it and in a recent interview she said   i don t really hide any feelings too much ', 'i think everyone knows this is my job here ', 'when i m on the courts or when i m on the court playing  i m a competitor and i want to beat every single person whether they re in the locker room or across the net so i m not the one to strike up a conversation about the weather and know that in the next few minutes i have to go and try to win a tennis match ', 'i m a pretty competitive girl ', 'i say my hellos  but i m not sending any players flowers as well ', 'uhm  i m not really friendly or close to many players ', 'i have not a lot of friends away from the courts  ', 'when she said she is not really close to a lot of players  is that something strategic that she is doing ', 'is it different on the men s tour than the women s tour ', ' no  not at all ', 'i thin

In [10]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [0]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [13]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
print(clean_sentences)

['maria sharapova basically friends tennis players wta tour', 'russian player problems openly speaking recent interview said really hide feelings much', 'think everyone knows job', 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match', 'pretty competitive girl', 'say hellos sending players flowers well', 'uhm really friendly close many players', 'lot friends away courts', 'said really close lot players something strategic', 'different men tour women tour', '', 'think sport mean friends everyone categorized tennis player going get along tennis players', 'think every person different interests', 'friends completely different jobs interests met different parts life', 'think everyone thinks tennis players greatest friends', 'ultimately tennis small part', 'many things interested', 'basel switzerland ap roger federer advanced th swiss indoors final career beating seventh seeded

In [14]:
'''# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip
! unzip glove*.zip'''

'# download pretrained GloVe word embeddings\n! wget http://nlp.stanford.edu/data/glove.6B.zip\n! unzip glove*.zip'

In [0]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


In [0]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)
  print(sentence_vectors)

In [20]:
len(sentence_vectors)

119

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [0]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [0]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 

Find the original article here https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/